# Notes

You might want to check https://julkaar9.github.io/walkalongs/orms/django/w3school/ to get  
basic an idea on how to solve these questions.  

Make sure to check the column names and foreign key relationships from server/customer_db/hospital_models.py.

Use `sql_to_df` to print the sql query results.  

Some insights on the equal function. 
1. Proper type casts might be required to get an equal result
2. Exact ordering of columns is not required for query results to be equal
3. Exact ordering of rows / tuples however are required.

In [1]:
import os

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from utils import *
from sqlparse import format

In [2]:
from django.db.models import (
    Q,
    F,
    Case,
    When,
    Count,
    Func,
    Min,
    Max,
    Sum,
    Avg,
    Value,
    OuterRef,
    Subquery,
    CharField,
)
from django.db.models.functions import Concat, Cast, Round, Length
from django.db import connection
from customer_db.models import Provinces, Patients, Doctors, Admissions

In [3]:
def print_sql(queryset):
    formatted = format(str(queryset.query), reindent=True)
    print(formatted)

## Testing some basic queries

In [4]:
sqlq = sql_raw("SELECT * FROM Patients LIMIT 3")
sql_to_df(sqlq)

,patient_id,first_name,last_name,gender,birth_date,city,allergies,height,weight,province_id
0,1,Cheryl,Mason,F,1978-05-14,Toronto,Peanuts,170,65,ON
1,2,Jason,Fleming,M,1990-08-02,Vancouver,Sulfa drugs,185,80,BC
2,3,Tiffany,McDonald,F,1985-03-27,Calgary,Lactose,155,58,AB


In [5]:
common_fields = [
    "first_name",
    "last_name",
    "gender",
    "birth_date",
    "city",
    "province_id",
    "allergies",
    "height",
    "weight",
]

In [6]:
ormq = Patients.objects.values(patient_id=F("id"), *common_fields)[:3]
orm_to_df(ormq)

,first_name,last_name,gender,birth_date,city,province_id,allergies,height,weight,patient_id
0,Cheryl,Mason,F,1978-05-14,Toronto,ON,Peanuts,170,65,1
1,Jason,Fleming,M,1990-08-02,Vancouver,BC,Sulfa drugs,185,80,2
2,Tiffany,McDonald,F,1985-03-27,Calgary,AB,Lactose,155,58,3


In [7]:
equal(sqlq, ormq)

Equal ✔️


# Example 
Show first_name of patient and admission date from admission table. Order by first_name and admission_date

In [8]:
qstr = """
SELECT
  p.first_name,
  admission_date
FROM admissions a
JOIN patients p on p.patient_id = a.patient_id
ORDER By p.first_name, admission_date
"""
sqlq = sql_raw(qstr)

In [9]:
ormq = Admissions.objects.values(
    "admission_date", first_name=F("patient__first_name")
).order_by("first_name", "admission_date")
equal(sqlq, ormq)
print_sql(ormq)

Equal ✔️
SELECT "admissions"."admission_date",
       "patients"."first_name" AS "first_name"
FROM "admissions"
INNER JOIN "patients" ON ("admissions"."patient_id" = "patients"."patient_id")
ORDER BY 2 ASC,
         "admissions"."admission_date" ASC


In [10]:
orm_to_df(ormq[:3])

,admission_date,first_name
0,2022-02-18,Abigail
1,2022-05-28,Abigail
2,2022-01-08,Adam


## Easy 1
Show first name, last name, and gender of patients who's gender is 'M'

In [11]:
qstr = """
SELECT
  first_name,
  last_name,
  gender
FROM Patients
WHERE gender = 'M';
"""
sqlq = sql_raw(qstr)

In [12]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [13]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 2
Show first name and last name of patients who does not have allergies. (null)

In [14]:
qstr = """
SELECT
  first_name,
  last_name
FROM Patients
WHERE allergies IS NULL;
"""
sqlq = sql_raw(qstr)

In [15]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [16]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 3
Show first name of patients that start with the letter 'C'

In [17]:
qstr = """
SELECT first_name
FROM Patients
WHERE first_name LIKE 'C%';
"""
sqlq = sql_raw(qstr)

In [18]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [19]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 4
Show first name and last name of patients that weight within the range of 100 to 120 (inclusive)
We will do 70 - 80

In [20]:
qstr = """
SELECT
  first_name,
  last_name
FROM Patients
WHERE weight BETWEEN 70 AND 80;
"""
sqlq = sql_raw(qstr)

In [21]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [22]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 5
Show first name and last name concatinated into one column to show their full name.

In [23]:
qstr = """
SELECT
  first_name || ' ' || last_name AS full_name
FROM patients;
"""
sqlq = sql_raw(qstr)

In [24]:
# sqlq =  sql_raw("SELECT CONCAT(first_name,' ', last_name) full_name FROM Patients;")
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [25]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 6
Show first name, last name, and the **full** province name of each patient.

Example: 'Ontario' instead of 'ON'

In [26]:
qstr = """
SELECT
  first_name,
  last_name,
  province_name
FROM Patients p
  INNER JOIN province_names pn ON p.province_id = pn.province_id;
"""
sqlq = sql_raw(qstr)

In [27]:
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [28]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 7
Show how many patients have a birth_date with 2010 as the birth year.
We will do 1995

In [29]:
# sqlq =  sql_raw("SELECT COUNT(*) birth_year FROM Patients WHERE YEAR(birth_date) = 2010;")

# for sqlite (Notice the '1995' instead of 1995)
qstr = """
SELECT COUNT(*) AS birth_count
FROM Patients
WHERE strftime('%Y', birth_date) = '1995';
"""
sqlq = sql_raw(qstr)

In [30]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [31]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 8
Show the first_name, last_name, and height of the patient with the greatest height.

In [32]:
qstr = """
SELECT
  first_name,
  last_name,
  Max(height) AS max_height
FROM patients;
"""
sqlq = sql_raw(qstr)

In [33]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [34]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 9
Show all columns for patients who have one of the following patient_ids:
1,45,534,879,1000
We will use 1,45,53,87,100

In [35]:
qstr = """
SELECT
  *
FROM patients
WHERE patient_id IN (1, 45, 53, 87, 100);
"""
sqlq = sql_raw(qstr)

In [36]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [37]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 10
Show the total number of admissions

In [38]:
sqlq = sql_raw("SELECT COUNT(*) AS total FROM admissions;")
ormq = Patients.objects.values("id")
equal(sqlq, ormq)

Unequal ❌


In [39]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 11
Show all the columns from admissions where the patient was admitted and discharged on the same day.

In [40]:
qstr = """
SELECT *
FROM admissions
WHERE admission_date = discharge_date;
"""
sqlq = sql_raw(qstr)

In [41]:
admission_fields = [
    "id",
    "patient_id",
    "admission_date",
    "discharge_date",
    "diagnosis",
    "attending_doctor_id",
]

In [42]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [43]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 12
Show the patient id and the total number of admissions for patient_id 579.
We will do 57

In [44]:
qstr = """
SELECT
  patient_id,
  Count(patient_id) admissions
FROM admissions
WHERE patient_id = 57;
"""
sqlq = sql_raw(qstr)

In [45]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [46]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 13
Based on the cities that our patients live in, show unique cities that are in province_id 'NS'?

In [47]:
qstr = """
SELECT DISTINCT city
FROM patients
WHERE province_id = 'NS';
"""
sqlq = sql_raw(qstr)

In [48]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [49]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 14
Write a query to find the first_name, last name and birth date of patients who has height greater than 160 and weight greater than 70

In [50]:
qstr = """
SELECT
  first_name,
  last_name,
  birth_date
FROM patients
WHERE height > 160 AND weight > 70;
"""
sqlq = sql_raw(qstr)

In [51]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [52]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 15
Write a query to find list of patients first_name, last_name, and allergies from Hamilton where allergies are not null.
We will use Toronto

In [53]:
qstr = """
SELECT
  first_name,
  last_name,
  allergies
FROM patients
WHERE
  city = 'Toronto'
  AND allergies IS NOT NULL;
"""
sqlq = sql_raw(qstr)

In [54]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [55]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3


## Easy 16
Based on cities where our patient lives in, write a query to display the list of unique city starting with a vowel (a, e, i, o, u). Show the result order in ascending by city.

In [56]:
# You can use city LIKE '[aeiou]%'
# We are doing case insensitve matching here.
qstr = """
SELECT DISTINCT city
FROM patients
WHERE
  city LIKE 'a%'
  OR city LIKE 'e%'
  OR city LIKE 'i%'
  OR city LIKE 'o%'
  OR city LIKE 'u%'
ORDER BY city;
"""
sqlq = sql_raw(qstr)

In [57]:
# ["a", "e", "i", "o", "u"]:
ormq = Patients.objects.values("id")
equal(sqlq, ormq)
print_sql(ormq)

Unequal ❌
SELECT "patients"."patient_id"
FROM "patients"


In [58]:
orm_to_df(ormq[:3])

,id
0,1
1,2
2,3
